In [1]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn

from util.ZigZag import ZigZag
from util.BuildData import TrainingSet_NN_Prices, TrainingSet_NN_Logret
from util.BuildData import TrainingSet_ML_Prices, TrainingSet_ML_Logret
from util.CalcReturns import CalcReturns

from sklearn import svm, preprocessing


In [2]:
%matplotlib qt

In [3]:
def recurrent_neural_network_model(x):

    layer = {'w' : tf.Variable(tf.random_normal([num_hidden, n_classes])),
             'b' : tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.unstack(x, timesteps, 1)    
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)    
    outs, state = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)        
    y = tf.add(tf.matmul(outs[-1], layer['w']), layer['b'], name='nn')
    
    return y

In [4]:
def train_and_save_nn(x,y):
    nn = recurrent_neural_network_model(x)
    
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = nn, labels = y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(cost)   
    #optimizer = tf.train.AdamOptimizer(learning_rate = 0.0006).minimize(cost)   
    
    correct = tf.equal(tf.argmax(nn,1), tf.argmax(y_train, 1), name='correct')
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name = 'accuracy')
    
    saver = tf.train.Saver()
    
    with tf.Session() as session:
        session.run(tf.initialize_all_variables())

        epoch_loss = 0
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(int(len(X_train)/batch_size)):
                
                batch_x = X_train[i*batch_size : (i+1)*batch_size]
                batch_y = Y_train[i*batch_size : (i+1)*batch_size]
               
                batch_x = batch_x.reshape((batch_size, timesteps, num_input))
                
                _, c = session.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
                epoch_loss += c
                
            if epoch % 100 == 0:
                print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
        
        
        
                
        saver.save(session, 'data/fxnn.ckpt')
        
        print('Last epoch loss: ', epoch_loss)
        return nn, correct, accuracy

In [5]:
def test_nn(x_test, y_test):
    with tf.Session() as session:
        saver = tf.train.import_meta_graph('data/fxnn.ckpt.meta')
        saver.restore(session, tf.train.latest_checkpoint('data/'))
        
        graph    = tf.get_default_graph()
        correct  = graph.get_tensor_by_name('correct:0')
        accuracy = graph.get_tensor_by_name('accuracy:0')
        
        x = graph.get_tensor_by_name('x:0')
        y = graph.get_tensor_by_name('y:0')

        x_test = x_test.reshape((t_count, timesteps, num_input))
        
        r = tf.cast(correct, 'float').eval({x:x_test, y:y_test})
        
        test_buy  = 0.0
        test_sell = 0.0
        test_hold = 0.0

        correct_buy  = 0.0
        correct_sell = 0.0
        correct_hold = 0.0

        incorrect_buy  = 0.0
        incorrect_sell = 0.0
        incorrect_hold = 0.0
        
        for i in range(len(r)):
          
            if y_test[i][2] == 1:
                test_buy += 1.0;
            elif y_test[i][1] == 1:
                test_hold += 1.0;
            elif y_test[i][0] == 1:
                test_sell += 1.0;
                
            if r[i] == 1:
                if y_test[i][2] == 1:
                    correct_buy += 1.0;
                elif y_test[i][1] == 1:
                    correct_hold += 1.0;
                elif y_test[i][0] == 1:
                    correct_sell += 1.0;
            else:
                if y_test[i][2] == 1:
                    incorrect_buy += 1.0;
                elif y_test[i][1] == 1:
                    incorrect_hold += 1.0;
                elif y_test[i][0] == 1:
                    incorrect_sell += 1.0;
        
        print('Accuracy:',accuracy.eval({x_train:x_test, y_train:y_test}))
        
        print( "Test buy:  ", test_buy  )
        print( "Test sell: ", test_sell )
        print( "Test hold: ", test_hold )

        print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/test_buy)*100 )
        print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/test_sell)*100 )
        print( "Correct hold: ", correct_hold, "Incorrect hold: ", incorrect_hold, "Accuracy: ", (correct_hold/test_hold)*100 )

In [6]:
n_classes  = 3
#n_classes  = 1
step = 1
timesteps = 1
num_input = 1

#step == num_input * timesteps

hm_epochs  = 1000
num_hidden = 240
batch_size = 128
t_count = 1024


In [7]:
source = pd.read_csv('EURUSD1.csv', header=0) # source CSV
prices = np.array(source.Close) # close prices

T, S = TrainingSet_NN_Logret(prices = prices, minsize = 0.0064, lag = step, scale = True)
#T, S = TrainingSet_ML_Prices(prices = prices, minsize = 0.0064, lag = step, scale = True)

X_train = T['input'][:-t_count]
Y_train = T['label'][:-t_count]

X_test = T['input'][-t_count:]
Y_test = T['label'][-t_count:]

Buys: 1899; Sells: 2097; Holds: 10371; Count: 14367


In [8]:
x_train = tf.placeholder('float', [None, timesteps, num_input], name='x')
y_train = tf.placeholder('float', [None, n_classes], name='y')

train_and_save_nn(x_train, y_train)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 0, 'completed out of', 1000, 'loss', 79.078848004341125)
('Epoch', 100, 'completed out of', 1000, 'loss', 49.184270098805428)
('Epoch', 200, 'completed out of', 1000, 'loss', 49.184199407696724)
('Epoch', 300, 'completed out of', 1000, 'loss', 49.184194564819336)
('Epoch', 400, 'completed out of', 1000, 'loss', 49.184190362691879)
('Epoch', 500, 'completed out of', 1000, 'loss', 49.184187009930611)
('Epoch', 600, 'completed out of', 1000, 'loss', 49.184182375669479)
('Epoch', 700, 'completed out of', 1000, 'loss', 49.184177458286285)
('Epoch', 800, 'completed out of', 1000, 'loss', 49.184174194931984)
('Epoch', 900, 'completed out of', 1000, 'loss', 49.18417039513588)
('Last epoch loss: ', 49.184166088700294)


(<tf.Tensor 'nn:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'correct:0' shape=(?,) dtype=bool>,
 <tf.Tensor 'accuracy:0' shape=() dtype=float32>)

In [9]:
test_nn(X_test, Y_test)

('Accuracy:', 0.81152344)
('Test buy:  ', 124.0)
('Test sell: ', 69.0)
('Test hold: ', 831.0)
('Correct buy:  ', 0.0, 'Incorrect buy:  ', 124.0, 'Accuracy: ', 0.0)
('Correct sell: ', 0.0, 'Incorrect sell: ', 69.0, 'Accuracy: ', 0.0)
('Correct hold: ', 831.0, 'Incorrect hold: ', 0.0, 'Accuracy: ', 100.0)


In [10]:
from scipy.linalg import hankel
L = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
hankel(L[0:1], L[0:]).T

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10]])